# **Homework 1 - Text Mining**

## Group Members:
1. **Tarang Kadyan**  
   <tarang.kadyan@bse.eu>

2. **Oriol Gelabert**  
   <oriol.gelabert@bse.eu>

3. **Enzo Infantes**  
   <enzo.infantes@bse.eu>

<img src='https://upload.wikimedia.org/wikipedia/commons/4/41/BSE_primary_logo_color.jpg' width=300 />

# **1. Libraries**

In [1]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
import os
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options

## Preferences
This contains past preferences such as passwords, cookie acceptance etc.

In [5]:
def ffx_preferences(dfolder, download=False):
    '''
    Sets the preferences of the firefox browser: download path.
    '''
    profile = webdriver.FirefoxProfile()
    # set download folder:
    profile.set_preference("browser.download.dir", dfolder) # you can predefine where you wanna store things in case its needed
    profile.set_preference("browser.download.folderList", 2) # 0 means to download to the desktop, 1 means to download to the default "Downloads" directory, 2 means to use the directory
    profile.set_preference("browser.download.manager.showWhenStarting", False) #I dont wanna see a pop up for each download so i swith it off
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk",
                           "application/msword,application/rtf, application/csv,text/csv,image/png ,image/jpeg, application/pdf, text/html,text/plain,application/octet-stream")

    # profile.install_addon('/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse_2025/booking/booking/ublock_origin-1.55.0.xpi')
    # profile.add_extension('/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse/booking/booking/ublock_origin-1.55.0.xpi')


    # this allows to download pdfs automatically
    if download:
        profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf,application/x-pdf")
        profile.set_preference("pdfjs.disabled", True) #dont want the pdf viewer to open


    options = Options()
    options.profile = profile
    options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe" #path to firefox (change it to your path)

    return options


def start_up(link, dfolder, geko_path,donwload=True):
    # geko_path='/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse/booking/geckodriver'
    # download_path='./downloads'
    os.makedirs(dfolder, exist_ok=True)

    options = ffx_preferences(dfolder,donwload)
    service = Service(geko_path)
    browser = webdriver.Firefox(service=service, options=options)
    # Enter the website address here
    browser.get(link)
    time.sleep(5)  # Adjust sleep time as needed
    return browser


def check_and_click(browser, xpath, type):
    '''
    Function that checks whether the object is clickable and, if so, clicks on
    it. If not, waits one second and tries again.
    '''
    ck = False
    ss = 0
    while ck == False:
        ck = check_obscures(browser, xpath, type)
        time.sleep(1)
        ss += 1
        if ss == 15:
            # warn_sound()
            # return NoSuchElementException
            ck = True
            # browser.quit()

def check_obscures(browser, xpath, type):
    '''
    Function that checks whether the object is being "obscured" by any element so
    that it is not clickable. Important: if True, the object is going to be clicked!
    '''
    try:
        if type == "xpath":
            browser.find_element('xpath',xpath).click()
        elif type == "id":
            browser.find_element('id',xpath).click()
        elif type == "css":
            browser.find_element('css selector',xpath).click()
        elif type == "class":
            browser.find_element('class name',xpath).click()
        elif type == "link":
            browser.find_element('link text',xpath).click()
    except (ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException) as e:
        print(e)
        return False
    return True

Open Booking.com website to start our search:

In [7]:
# lets open booking:
dfolder= os.getcwd()
geko_path = os.path.join(dfolder, 'geckodriver.exe')
link='https://www.booking.com/index.es.html'


browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path)

Reject Cookies: The following cell closes Rejects the cookies, closing the banner and avoiding possible interference when scrapping.

In [8]:
path_cookies='//button[@id="onetrust-reject-all-handler"]'
cookies= browser.find_elements('xpath',path_cookies)
cookies[0].click()

Close Google Log-in

Google Log-In Pop-Up is inside an Iframe object, so we should first acces to the iframe and latter to the element we are willing to click, in this case the close button of this pop-up.

In [9]:
iframe_google = browser.find_elements(By.TAG_NAME,'iframe')[0] #we find the iframe object
browser.switch_to.frame(iframe_google) #switch to acces the iframe

In [10]:
close_log_in = browser.find_element(By.CSS_SELECTOR, '#close') # find the close button element
close_log_in.click() #click on it to close the pop-up
browser.switch_to.default_content() #switch back the browser to default to exit the iframe and continue web-scrapping

Find ''Where are you Going?'' Box and click on it:

In [11]:
browser.find_element(by=By.XPATH,value='//*[@id=":rh:"]').click()

We initially will search hotels in Barcelona, so we should input this value on the searchbox:

In [12]:
place = 'Barcelona'
search1 = browser.find_element(by=By.NAME,value='ss')
search1.send_keys(place)

Now, we select the dates of the event we wnat to track, in our case MWC. This event is hosted between 3-6 March, so our initial serach will be on the week of 1-8 March.

First we open the selector of dates:

In [13]:
css='button.ebbedaf8ac:nth-child(2) > span:nth-child(1)'

browser.find_element('css selector', css).click()

Advance to next month, to be able to select March dates:

In [14]:
path_next_month='//button[@class="a83ed08757 c21c56c305 f38b6daa18 d691166b09 f671049264 f4552b6561 dc72a8413c f073249358"]'
next_month= browser.find_elements('xpath',path_next_month)
next_month[0].click()

We acces all dates that are now visible, including both February and March:

In [15]:
id = "calendar-searchboxdatepicker"
span_class = "cf06f772fa ef091eb985"
path = f'//div[@id="{id}"]//table[@class="eb03f3f27f"]//tbody//td[@class="b80d5adb18"]//span[@class="{span_class}"]'
dates = browser.find_elements('xpath',path)
for date in dates:
    print(date.get_attribute("data-date"))

2025-02-01
2025-02-02
2025-02-03
2025-02-04
2025-02-05
2025-02-06
2025-02-07
2025-02-08
2025-02-09
2025-02-10
2025-02-11
2025-02-12
2025-02-13
2025-02-14
2025-02-15
2025-02-16
2025-02-17
2025-02-18
2025-02-19
2025-02-20
2025-02-21
2025-02-22
2025-02-23
2025-02-24
2025-02-25
2025-02-26
2025-02-27
2025-02-28
2025-03-01
2025-03-02
2025-03-03
2025-03-04
2025-03-05
2025-03-06
2025-03-07
2025-03-08
2025-03-09
2025-03-10
2025-03-11
2025-03-12
2025-03-13
2025-03-14
2025-03-15
2025-03-16
2025-03-17
2025-03-18
2025-03-19
2025-03-20
2025-03-21
2025-03-22
2025-03-23
2025-03-24
2025-03-25
2025-03-26
2025-03-27
2025-03-28
2025-03-29
2025-03-30
2025-03-31


In [16]:
from_day = input('Day from which you want to look for (In march)') #important two digits
to_day = input('Day until which you want to look for (In march)')


for date in dates:
    if date.get_attribute("data-date") == f"2025-03-{from_day}":
        date.click()
    if date.get_attribute("data-date") == f"2025-03-{to_day}":
        date.click()
        break

Bedroom selection

In [17]:
people_selection_path='//button[@class="a83ed08757 ebbedaf8ac dbaff8df6f ada2387af8"]'
people_selection= browser.find_elements('xpath',people_selection_path)
people_selection[0].click()

ONE ADULT MORE

In [18]:
path_cookies='//button[@class="a83ed08757 c21c56c305 f38b6daa18 d691166b09 ab98298258 bb803d8689 f4d78af12a"]'
cookies= browser.find_elements('xpath',path_cookies)
cookies[0].click()

ONE ADULT LESS

In [19]:
path_cookies='//button[@class="a83ed08757 c21c56c305 f38b6daa18 d691166b09 ab98298258 bb803d8689 e91c91fa93"]'
cookies= browser.find_elements('xpath',path_cookies)
cookies[0].click()

In [ ]:
my_xpath="/html/body/div[3]/div[2]/div/form/div/div[4]/button/span"

check_obscures(browser, my_xpath , type='xpath')
check_and_click(browser,my_xpath , type='xpath')